In [ ]:
from numpy.random import rand
import time
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
from sklearn import linear_model
from sklearn.neural_network import MLPRegressor
import tensorflow as tf
from scipy import stats
import torch
from torch.autograd import Variable
from keras.models import Sequential
from keras.layers import *

In [ ]:
test_time = 20.0

In [ ]:
w = rand()
b = rand()

In [ ]:
cols = ['scipy','statsmodel','numpy','sklearn','nn','keras nn','keras','tf','pytorch']
n = 100
index = []
for i in range(25):
    index.append(n)
    n *= 2
    
results = pd.DataFrame(columns=cols,index = index)

In [ ]:
# Statsmodels OLS
n = 100
start = time.time()
while time.time() <= start + test_time:
    x = np.linspace(0,100,n)
    y = w * x + b
    
    # Add Gaussian noise
    noise = np.random.normal(0,1.0,n); y += noise
    
    # scipy
    xc = sm.add_constant(x)
    t0 = time.time()
    model = sm.OLS(y,xc).fit()
    predictions = model.predict(xc)
    t1 = time.time()
    print(n,t1-t0)
    results['statsmodel'].loc[n] = t1 - t0
    
    n = n * 2 # Double number of data points

In [ ]:
# Numpy
n = 100
start = time.time()
while time.time() <= start + test_time:
    x = np.linspace(0,100,n)
    y = w * x + b
    
    # Add Gaussian noise
    noise = np.random.normal(0,1.0,n)
    y += noise
    
    # numpy
    t0 = time.time()
    np.polyfit(x,y,1)
    t1 = time.time()
    print(n,t1-t0)
    results['numpy'].loc[n] = t1 - t0
    
    n = n * 2 # Double number of data points

In [ ]:
# sklearn
n = 100
start = time.time()
while time.time() <= start + test_time:
    x = np.linspace(0,100,n)
    y = w * x + b
    
    # Add Gaussian noise
    noise = np.random.normal(0,1.0,n)
    y += noise
    
    # numpy
    t0 = time.time()
    lm = linear_model.LinearRegression()
    lm.fit(x.reshape((n,1)),y)
    t1 = time.time()
    print(n,t1-t0)
    results['sklearn'].loc[n] = t1 - t0
    
    n = n * 2 # Double number of data points

In [ ]:
# Scikit-learn neural network
n = 100
start = time.time()
while time.time() <= start + test_time:
    x = np.linspace(0,100,n)
    y = w * x + b
    
    # Add Gaussian noise
    noise = np.random.normal(0,1.0,n)
    y += noise
    
    # Neural network
    t0 = time.time()
    nn = MLPRegressor(hidden_layer_sizes=((10,10)),activation='tanh',\
                      solver='lbfgs',max_iter=5000)
    nn.fit(x.reshape((n,1)),y)
    t1 = time.time()
    print(n,t1-t0)
    results['nn'].loc[n] = t1 - t0
    
    n = n * 2 # Double number of data points

In [ ]:
# Keras nn
n_inputs = 1
nodes = 10
skip = 2

# create neural network model
model = Sequential()
model.add(Dense(n_inputs, input_dim=n_inputs, activation='linear'))
model.add(Dense(nodes, activation='linear'))
model.add(Dense(nodes, activation='tanh'))
model.add(Dense(nodes, activation='tanh'))
model.add(Dense(nodes, activation='linear'))
model.add(Dense(1, activation='linear'))
model.compile(loss="mean_squared_error", optimizer="adam")

n = 100
start = time.time()
while time.time() <= start + test_time:
    x = np.linspace(0,100,n)
    y = w * x + b
    
    # Add Gaussian noise
    noise = np.random.normal(0,1.0,n)
    y += noise
    
    # Keras 
    t0 = time.time()
    model.fit(x,y,epochs=100,batch_size=int(n/10),verbose=0,shuffle=True)
    t1 = time.time()
    print(n,t1-t0)
    results['keras nn'].loc[n] = t1 - t0
    
    n = n * 2 * skip # Quadruple number of data points

In [ ]:
# Keras linear regression
model = Sequential()
model.add(Dense(1, input_dim=1, activation='relu'))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

n = 100
start = time.time()
while time.time() <= start + test_time:
    x = np.linspace(0,100,n)
    y = w * x + b
    
    # Add Gaussian noise
    noise = np.random.normal(0,1.0,n)
    y += noise
    
    # Keras 
    t0 = time.time()
    model.fit(x,y,epochs=100,batch_size=int(n/10),verbose=0,shuffle=True)
    t1 = time.time()
    print(n,t1-t0)
    results['keras'].loc[n] = t1 - t0
    
    n = n * 2 * skip # Quadruple number of data points

For Keras, TF, change input parameters such as epochs, batch size, nodes, number of layers, etc, to see how it affects time

In [ ]:
# Tensorflow
learning_rate = 0.01
epochs = 100
tf.get_logger().setLevel('ERROR')

# Weight and Bias variables
m = tf.Variable(tf.zeros(1), name='weight')
a = tf.Variable(tf.zeros(1), name='bias')

def predict(x):
    return x * m + a

def mse(y_true, y_pred):
    return tf.losses.mean_squared_error(y_true,y_pred)

def fit_TF(x,y):
    for epoch in range(1, epochs + 1):
    # Begin GradientTape and optimise
        with tf.GradientTape() as g:
            pred = predict(x)
            loss = mse(y, pred)

        # Compute dw, db
        gradients = g.gradient(loss, [m,a])

        # Update w and b
        optimizer.apply_gradients(zip(gradients, [m,a]))
    
    return (m.numpy(),a.numpy())

# SGD Optimizer
optimizer = tf.optimizers.SGD(learning_rate=learning_rate)

n = 100
start = time.time()

while time.time() <= start + test_time:
    x = np.linspace(0,100,n)
    y = w * x + b
    
    # Add Gaussian noise
    noise = np.random.normal(0,1.0,n)
    y += noise
    
    # TF
    t0 = time.time()
    fit_TF(x,y)
    t1 = time.time()
    print(n,t1-t0)
    results['tf'].loc[n] = t1 - t0
    
    n = n * 2 * skip

In [ ]:
# Scipy
n = 100
start = time.time()
while time.time() <= start + test_time:
    x = np.linspace(0,100,n)
    y = w * x + b
    
    # Add Gaussian noise
    noise = np.random.normal(0,1.0,n)
    y += noise
    
    # numpy
    t0 = time.time()
    stats.linregress(x,y)
    t1 = time.time()
    print(n,t1-t0)
    results['scipy'].loc[n] = t1 - t0
    
    n = n * 2 # Double number of data points

In [ ]:
# Pytorch
class linearRegression(torch.nn.Module):
    def __init__(self, inputSize, outputSize):
        super(linearRegression, self).__init__()
        self.linear = torch.nn.Linear(inputSize, outputSize)

    def forward(self, x):
        out = self.linear(x)
        return out
    
inputDim = 1        # takes variable 'x' 
outputDim = 1       # takes variable 'y'
learningRate = 0.01 
epochs = 100

model = linearRegression(inputDim, outputDim)
criterion = torch.nn.MSELoss() 
optimizer = torch.optim.SGD(model.parameters(), lr=learningRate)

def fit_pytorch(x,y):
    x = x.reshape(len(x),1)
    y = y.reshape(len(y),1)
    
    for epoch in range(epochs):
        # Converting inputs and labels to Variable
        inputs = Variable(torch.from_numpy(x).float())
        labels = Variable(torch.from_numpy(y).float())

        # Clear gradient buffers because we don't want any gradient
        #  from previous epoch to carry forward, dont want to 
        #  cummulate gradients
        optimizer.zero_grad()

        # get output from the model, given the inputs
        outputs = model(inputs)

        # get loss for the predicted output
        loss = criterion(outputs, labels)
        print(loss)
        # get gradients w.r.t to parameters
        loss.backward()

        # update parameters
        optimizer.step()
        
n = 100
start = time.time()

while time.time() <= start + test_time:
    x = np.linspace(0,100,n)
    y = w * x + b
    
    # Add Gaussian noise
    noise = np.random.normal(0,1.0,n)
    y += noise
    
    # TF
    t0 = time.time()
    fit_pytorch(x,y)
    t1 = time.time()
    print(n,t1-t0)
    results['pytorch'].loc[n] = t1 - t0
    
    n = n * 2 * skip

In [ ]:
plt.figure(figsize=(8,5))
plt.subplot(2,1,1)
plt.plot(results['keras'][::skip],'r-',label='keras linear')
plt.plot(results['tf'][::skip],'g-',label='tensorflow linear')
plt.plot(results['pytorch'][::skip],'b.-',label='pytorch linear')
plt.plot(results['statsmodel'],'r--',lw=2,label='statsmodel ols')
plt.plot(results['numpy'],'k-',lw=2,label='numpy polyfit')
plt.plot(results['sklearn'],'g:',lw=2,label='sklearn linear')
plt.plot(results['scipy'],'b-',lw=2,label='scipy linregress')
plt.xscale('log')
plt.ylabel('Train Time (sec)')
plt.legend()
xlim = plt.gca().get_xlim()
plt.ylim([0,test_time])
plt.grid()

plt.subplot(2,1,2)
plt.plot(results['nn'],'b-',label='sklearn nn')
plt.plot(results['keras nn'][::skip],'r:',label='keras nn')
plt.xscale('log')
plt.ylabel('Train Time (sec)')
plt.xlabel('Problem Size (samples)')
plt.legend()
plt.xlim(xlim)
plt.ylim([0,test_time])
plt.grid()
plt.savefig('timing_results.png',dpi=600)